In [1]:
using Agents , Random , InteractiveDynamics , CairoMakie 
using DrWatson: @dict
import Distributions: Categorical



mutable struct Agent2 <: AbstractAgent
    id::Int
    pos::NTuple{2,Float64}
    vel::NTuple{2,Float64}
    mass::Float64
    days_infected::Int # how many days have passed after infection
    status::Symbol # :S (susceptible), :I (infecious), :R (removed, immune), :Q (quarantine and infecious) :V (vaccinated and removed)
    days_quarantine::Int
    hygiene::Float64
end

function if_susceptible(agent::Agent2)
    return agent.status == :S
end


function symulation(;
    infection_period = 24 ,
    detection_time = 14 ,
    reinfection_probability = 0.01,
    isolated = 0.2, # 0 is nobody is isolated , 1 eve ryone is isolated
    interaction_radius = 0.01,
    dt = 1,
    speed = 0.0012,
    death_rate = 0.044,
    N = 1000,
    initial_infected = 5,
    seed = 1410,
    hygiene_min = 0.4,
    hygiene_max = 0.8,
    time_of_quarantine = 10,
    steps_per_day = 12,
    chance_to_go_quaratine = 0.5,
    symulation_time = 0,
    vaccine_per_day = 2
    )

    infection_period *= steps_per_day
    detection_time *=  steps_per_day
    time_of_quarantine *= steps_per_day

    properties = @dict(
        infection_period,
        detection_time,
        reinfection_probability,
        death_rate,
        dt,
        interaction_radius,
        time_of_quarantine,
        speed,
        chance_to_go_quaratine,
        steps_per_day,
        symulation_time,
        N,
        vaccine_per_day)

    space = ContinuousSpace((1,1), 0.02)
    model = ABM(Agent2,space, properties = properties, rng = MersenneTwister(seed))

    # Add agents to the model
   for idx in 1:N
        pos = Tuple(rand(model.rng, 2))
        status = idx > initial_infected ? :S : :I
        is_isolated = rand(model.rng, Categorical([isolated,1-isolated])) == 1 ? true : false 
        mass = is_isolated ? Inf : 1.0
        vel = is_isolated ? (0,0) : sincos(2pi* rand(model.rng,)) .* speed
        hygiene = (hygiene_max-hygiene_min)*rand(model.rng) + hygiene_min
        add_agent!(pos,model,vel, mass, 0, status, 0, hygiene)
    end
    return model
end

function transmit!(a,b,reinfection,model)
    if  count(a.status == :I for a in (a,b)) in (0,2)
        return
    end
    
    infected, healthy = a.status == :I ? (a,b) : (b,a)
    rand(model.rng) < (infected.hygiene + healthy.hygiene)/2 && return 

   
    if healthy.status == :R || healthy.status == :V
        if rand(model.rng) > reinfection 
            return
        end
    end

    healthy.status = :I
end

function vaccine!(a, model)
    a.status = :V   
end

function model_step!(model)
    r = model.interaction_radius
    for (a, b) in interacting_pairs(model, r, :nearest)
        transmit!(a,b,model.reinfection_probability,model)
        elastic_collision!(a, b, :mass)
    end
    
    if model.symulation_time % model.steps_per_day == 0
        for _ in 1:model.vaccine_per_day
            a = random_agent(model, if_susceptible)
            if a != nothing
                vaccine!(a, model)
            else
                model.symulation_time += 1
                return model
            end
        end
       
    end
    model.symulation_time += 1
    return model
end

function quarantine!(a,model)
    if a.status == :I && a.days_infected >= model.detection_time && rand(model.rng) < model.chance_to_go_quaratine / model.steps_per_day
        a.status = :Q
        a.vel = (0,0)
        a.mass = Inf

    end
end

function quarantine_end!(a,model)
    if a.days_quarantine == model.time_of_quarantine
        a.days_quarantine = 0
        recover_or_die!(a,model) 
        a.vel = sincos(2pi* rand(model.rng,)) .* model.speed
        a.mass = 1.0
    end
    return model
end

function recover_or_die!(agent, model)
    if agent.days_infected >= model.infection_period
        if rand(model.rng) <= model.death_rate
            if agent == nothing
                print("nothing")
            end
            kill_agent!(agent, model)
        else 
            agent.status = :R
            agent.days_infected = 0
        end
    end
    return model
end

function agent_step!(agent,model)
    move_agent!(agent, model, model.dt)
    agent.status in (:I,:Q) ? agent.days_infected +=1 : agent.days_infected = 0
    agent.status == :Q ? agent.days_quarantine +=1 : agent.days_quarantine = 0
    recover_or_die!(agent, model)
    quarantine!(agent,model)
    quarantine_end!(agent,model)
end
sir_colors(a) = a.status == :S ? "#000000" : a.status == :I ? "#ff0000" : a.status == :Q ? "#00FFFF" : a.status == :V ? "#993399" : "#00FF00"


sir_colors (generic function with 1 method)

In [2]:
import CairoMakie
sir_model = symulation(isolated = 0 )

abm_video("symolation_without_isolation.mp4",
    sir_model,
    agent_step!,
    model_step!,
    title = " Symulation",
    ac = sir_colors,
    frames = 500 , spf = 2, framerate = 25)



In [3]:
using JLD2
#an aggregating functions
susceptible(x) = count(i == :S for i in x)
recovered(x) = count(i == :R for i in x)
infected(x) = count(i in [:I,:Q] for i in x)

sir_model1 = symulation(isolated = 0.8)
sir_model2 = symulation(isolated = 1)
sir_model3 = symulation(isolated = 0.4)
sir_model4 = symulation(isolated = 0)

adata = [(:status, susceptible), (:status, recovered), (:status, infected)]
Threads.@spawn global   agents1_df, = run!(sir_model1, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agents2_df, = run!(sir_model2, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agents3_df, = run!(sir_model3, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agents4_df, = run!(sir_model4, agent_step!, model_step!, 1000; adata)

sir_modela = symulation(isolated = 0.8, hygiene_max = 0.8, hygiene_min = 0.7)
sir_modelb = symulation(isolated = 0.8, hygiene_max = 0.8, hygiene_min = 0.5)
sir_modelc = symulation(isolated = 0.8, hygiene_max = 0.8, hygiene_min = 0.3)
sir_modeld = symulation(isolated = 0.8, hygiene_max = 0.8, hygiene_min = 0.0)


Threads.@spawn global   agentsa, = run!(sir_modela, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agentsb, = run!(sir_modelb, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agentsc, = run!(sir_modelc, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agentsd, = run!(sir_modeld, agent_step!, model_step!, 1000; adata)


sir_model1a = symulation(isolated = 0.8, hygiene_max = 1.0,hygiene_min = 0.1)
sir_model1b = symulation(isolated = 0.8, hygiene_max = 0.8,hygiene_min = 0.1)
sir_model1c = symulation(isolated = 0.8, hygiene_max = 0.5,hygiene_min = 0.1)
sir_model1d = symulation(isolated = 0.8, hygiene_max = 0.2,hygiene_min = 0.1)

Threads.@spawn global   agents1a, = run!(sir_model1a, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agents1b, = run!(sir_model1b, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agents1c, = run!(sir_model1c, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agents1d, = run!(sir_model1d, agent_step!, model_step!, 1000; adata)


sir_modelk1 = symulation(chance_to_go_quaratine = 0, isolated = 0.6)
sir_modelk2 = symulation(chance_to_go_quaratine = 12, isolated = 0.6)

Threads.@spawn global   agentsk1, = run!(sir_modelk1, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   agentsk2, = run!(sir_modelk2, agent_step!, model_step!, 1000; adata)


density_model_1000 = symulation(N = 1000)
density_model_800 = symulation(N = 800)
density_model_600 = symulation(N = 600)
density_model_400 = symulation(N = 400)
density_model_200 = symulation(N = 200)

Threads.@spawn global   density_1000, = run!(density_model_1000, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   density_800, = run!(density_model_800, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   density_600, = run!(density_model_600, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   density_400, = run!(density_model_400, agent_step!, model_step!, 1000; adata)
Threads.@spawn global   density_200, = run!(density_model_200, agent_step!, model_step!, 1000; adata)

Task (done) @0x0000000019d50b00

In [4]:
using JLD2, DataFrames , Plots
@save "izolacja.jld2" agents1_df agents2_df agents3_df agents4_df
@save "higiena_min.jld2" agentsa agentsb agentsc agentsd
@save "higiena_max.jld2" agents1a agents1b agents1c agents1d
@save "kwarantanna.jld2" agentsk1 agentsk2 
@save "gestosc.jld2" density_1000 density_800 density_600 density_400 density_200
println("Koniec")

Koniec


In [5]:
using JLD2, DataFrames , Plots
@load "izolacja.jld2"
plotly()
x = agents3_df.step
p1 = Plots.plot(x, agents3_df[:, aggname(:status, infected)],
    linecolor = "red",
    lw = 1.5,
    xlabel = "time",
    ylabel = "population",
    label = "infected",
    title = "Pandemic development for simulation",
    legend = :outerright)
Plots.plot!(x, agents3_df[:, aggname(:status, recovered)],
    linecolor = :green,
    lw = 1.5,
    label = "recovered")
Plots.plot!(x, agents3_df[:, aggname(:status, susceptible)],
    linecolor = :blue,
    lw = 1.5,
    label = "susceptible")



┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots C:\Users\mkarc\.julia\packages\Plots\SVksJ\src\backends.jl:372


<!DOCTYPE html>
 
 
 Plots.jl

In [6]:
using JLD2, DataFrames , Plots
@load "izolacja.jld2"
x = agents1_df.step
plotly()
Plots.plot(x, agents2_df[:, aggname(:status, infected)],
    linecolor = "#00CC00",
    lw = 1.5,
    ylabel = "Infected",
    xlabel = "time",
    label = "isolated=1.0",
    title = "Impact of isolation on the number of infected",
    legend = :outerright)
Plots.plot!(x, agents1_df[:, aggname(:status, infected)],
    linecolor = "#FFD700",
    lw = 1.5,
    label = "isolated=0.8")
Plots.plot!(x, agents3_df[:, aggname(:status, infected)],
    linecolor = "#FF0000",
    lw = 1.5,
    label = "isolated=0.4")
Plots.plot!(x, agents4_df[:, aggname(:status, infected)],
    linecolor = "#8B0000",
    lw = 1.5,
    label = "isolated=0.0")

<!DOCTYPE html>
 
 
 Plots.jl

In [7]:
using JLD2, DataFrames , Plots
@load "higiena_min.jld2"

x = agentsd.step
plotly()
p = Plots.plot(x, agentsd[:, aggname(:status, infected)],
    linecolor = "#FFD700",
    lw = 1.5,
    ylabel = "Infected",
    xlabel = "time",
    label = "min=0.0, max=0.8",
    title = "Impact of hygiene on the number of infection",
    legendtitle = "hygiene",
    legend = :left)
p =Plots.plot!(x, agentsc[:, aggname(:status, infected)],
    linecolor = "#FF6600",
    lw = 1.5,
    label = "min=0.3, max=0.8")
p = Plots.plot!(x, agentsb[:, aggname(:status, infected)],
    linecolor = "#CC0000",
    lw = 1.5,
    label = "min=0.5, max=0.8")
p =Plots.plot!(x, agentsa[:, aggname(:status, infected)],
    linecolor = "#660033",
    lw = 1.5,
    label = "min=0.7, max=0.8")

display(p)

<!DOCTYPE html>
 
 
 Plots.jl

In [8]:
using JLD2, DataFrames , Plots
@load "higiena_max.jld2"

x = agents1d.step
plotly()
Plots.plot(x, agents1d[:, aggname(:status, infected)],
    linecolor = "#FFD700",
    lw = 1.5,
    ylabel = "Infected",
    xlabel = "time",
    label = "max=0.2, min=0.1",
    title = "Impact of hygiene on the number of infection",
    legendtitle = "hygiene",
    legend = :left)
Plots.plot!(x, agents1c[:, aggname(:status, infected)],
    linecolor = "#FF6600",
    lw = 1.5,
    label = "max=0.5, min=0.1")
Plots.plot!(x, agents1b[:, aggname(:status, infected)],
    linecolor = "#CC0000",
    lw = 1.5,
    label = "max=0.8, min=0.1")
Plots.plot!(x, agents1a[:, aggname(:status, infected)],
    linecolor = "#660033",
    lw = 1.5,
    label = "max=1.o, min=0.1")

<!DOCTYPE html>
 
 
 Plots.jl

In [9]:
using JLD2, DataFrames , Plots
@load "kwarantanna.jld2"
x = agentsk1.step
plotly()
pk = Plots.plot(x, agentsk1[:, aggname(:status, infected)],
    linecolor = "red",
    lw = 1.5,
    title = "time of quarantine = 0",
#     xlabel = "time",
    ylabel = "population",
    label = "infected",
    ylims = (0,700),)
Plots.plot!(x, agentsk1[:, aggname(:status, recovered)],
    linecolor = :green,
    lw = 1.5,
    label = "recovered")
Plots.plot!(x, agentsk1[:, aggname(:status, susceptible)],
    linecolor = :blue,
    lw = 1.5,
    label = "susceptible")

pk2 = Plots.plot(x, agentsk2[:, aggname(:status, infected)],
    linecolor = "red",
    lw = 1.5,
    title = "time of quarantine = 10",
    xlabel = "time",
    ylabel = "population",
    label = "infected",
    ylims = (0,700),)
Plots.plot!(x, agentsk2[:, aggname(:status, recovered)],
    linecolor = :green,
    lw = 1.5,
    label = "recovered")
Plots.plot!(x, agentsk2[:, aggname(:status, susceptible)],
    linecolor = :blue,
    lw = 1.5,
    label = "susceptible"
    )

Plots.plot(pk, pk2, layout = (2, 1), legend= :false)


<!DOCTYPE html>
 
 
 Plots.jl

In [10]:
using JLD2, DataFrames , Plots
@load "izolacja.jld2"
x = agents1_df.step
plotly()

Plots.plot(x, agents2_df[:, aggname(:status, infected)],
    linecolor = "#00CC00",
    lw = 1.5,
    ylabel = "Infected",
    xlabel = "time",
    label = "isolated=1.0",
    title = "Impact of isolation on the number of infected")
Plots.plot!(x, agents1_df[:, aggname(:status, infected)],
    linecolor = "#FFD700",
    lw = 1.5,
    label = "isolated=0.8")
Plots.plot!(x, agents3_df[:, aggname(:status, infected)],
    linecolor = "#FF0000",
    lw = 1.5,
    label = "isolated=0.4")
Plots.plot!(x, agents4_df[:, aggname(:status, infected)],
    linecolor = "#8B0000",
    lw = 1.5,
    label = "isolated=0.0")

<!DOCTYPE html>
 
 
 Plots.jl

In [11]:
using JLD2, DataFrames , Plots
@load "izolacja.jld2"
x = Int16(floor(agents1_df.step[end] / 12))
plotly()
dy = [agents1_df[:,aggname(:status, infected)][12*(i+1)] - agents1_df[:, aggname(:status, infected)][12* i] for i in 1:(x-1)]
Plots.plot(1:(x-1) ,dy,linestyle = :dash)
Plots.scatter!(1:(x-1) ,dy)

<!DOCTYPE html>
 
 
 Plots.jl

In [12]:
using JLD2, DataFrames , Plots
@load "gestosc.jld2"
x = density_1000.step
plotly()
Plots.plot(x, density_1000[:, aggname(:status, infected)] / sum(density_1000[1,2:4]),
    ylabel="Part of population which is infected",
    xlabel="time",
    label="1000 Population")
Plots.plot!(x, density_800[:, aggname(:status, infected)] / sum(density_800[1,2:4]),label="800 Population")
Plots.plot!(x, density_600[:, aggname(:status, infected)] / sum(density_600[1,2:4]),label="600 Population")
Plots.plot!(x, density_400[:, aggname(:status, infected)] / sum(density_400[1,2:4]),label="400 Population")
Plots.plot!(x, density_200[:, aggname(:status, infected)] / sum(density_200[1,2:4]),label="200 Population")

<!DOCTYPE html>
 
 
 Plots.jl